# 分层建模：地震警报级别预测

本 Notebook 采用分层建模（Hierarchical Modeling）策略来解决四分类问题。灵感来源于对特征与标签关系的深入分析，特别是 `depth` 特征在区分 `label 0` 时的独特性。

**分析流程:**
1.  **第一阶段：分离 `label 0`**
    *   **目标**: 训练一个简单的二元分类器，精确地识别出 `label 0`。
    *   **数据**: 使用 **原始特征**，不做任何复杂处理，以利用 `depth` 等特征的原始分布信息。
    *   **模型**: 使用逻辑回归（Logistic Regression），因为它速度快且效果好。

2.  **第二阶段：区分 `label 1, 2, 3`**
    *   **目标**: 在排除了 `label 0` 的数据集上，训练一个更强大的多分类模型。
    *   **数据**: 对剩余数据进行完整的 **预处理、特征工程和SMOTE**，以最大化模型对 `label 1, 2, 3` 之间细微差别的捕捉能力。
    *   **模型**: 使用随机森林（RandomForestClassifier）。

3.  **合并预测与生成提交文件**
    *   将两个阶段的预测结果合并，生成最终的提交文件。

In [1]:
# 导入所有需要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize

# 模型与评估
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

# 不平衡处理
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
# 加载数据
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print("✅ Data loaded successfully!")
print("Training data shape:", df_train.shape)
print("Test data shape:", df_test.shape)

✅ Data loaded successfully!
Training data shape: (909, 7)
Test data shape: (391, 6)


## 第一阶段：使用原始数据分离 `label 0`

In [ ]:
print("🚀 Starting Stage 1: Isolating Label 0")

# 1. 准备数据 (使用原始特征)
features_raw = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']
X_stage1 = df_train[features_raw]
y_stage1 = (df_train['label'] == 0).astype(int) # 1 if label is 0, else 0

print("\nClass distribution for Stage 1:")
print(y_stage1.value_counts())

# 2. 选择并训练模型 (随机森林)
# 随机森林同样支持 class_weight='balanced' 来处理类别不平衡
stage1_model = RandomForestClassifier(
    n_estimators=500,          # 增加树的数量以提高稳定性
    max_depth=10,              # 限制深度以防止过拟合
    random_state=42,
    class_weight='balanced',
    n_jobs=-1                  # 使用所有CPU核心
)

# 3. 交叉验证评估模型性能
cv_scores = cross_val_score(stage1_model, X_stage1, y_stage1, cv=10, scoring='f1_macro')
print(f"\nStage 1 Model (RandomForest) CV Macro-F1 Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

# 4. 在全部训练数据上训练最终的第一阶段模型
stage1_model.fit(X_stage1, y_stage1)
print("\n✅ Stage 1 model trained successfully!")

# 查看特征重要性 (随机森林的 Gini importance)
# Feature importances show the contribution of each feature to the model's decisions
importance_df = pd.DataFrame(stage1_model.feature_importances_, index=features_raw, columns=['Importance'])
print("\nFeature importances for identifying Label 0:")
print(importance_df.sort_values('Importance', ascending=False))
print("-> 正如EDA所料, 'depth' 是区分 label 0 的最重要特征。")

🚀 Starting Stage 1: Isolating Label 0

Class distribution for Stage 1:
label
0    674
1    235
Name: count, dtype: int64

Stage 1 Model CV Macro-F1 Score: 0.8524 (+/- 0.1080)

✅ Stage 1 model trained successfully!

Feature coefficients for identifying Label 0:
           Coefficient
sig          -0.001651
depth        -0.001699
magnitude    -0.354804
cdi          -0.480340
mmi          -1.710632
-> 正如EDA所料, 'depth' 是区分 label 0 的最重要正向特征。


## 第二阶段：处理数据并区分 `label 1, 2, 3`

In [ ]:
print("🚀 Starting Stage 2: Classifying Labels 1, 2, and 3")

# 1. 创建第二阶段的数据集
df_stage2_train = df_train[df_train['label'] != 0].copy()
print(f"Number of samples for Stage 2: {len(df_stage2_train)}")
print("Class distribution for Stage 2:")
print(df_stage2_train['label'].value_counts())

# 2. 应用完整的预处理和特征工程流程
# a. Winsorize 缩尾处理
feature_cols = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']
for col in feature_cols:
    df_stage2_train[col] = winsorize(df_stage2_train[col], limits=[0.01, 0.05])

# b. 对 depth 进行对数变换
df_stage2_train['depth'] = np.log1p(df_stage2_train['depth'])

# c. 标准化
scaler = StandardScaler()
numerical_features = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']
df_stage2_train[numerical_features] = scaler.fit_transform(df_stage2_train[numerical_features])

# d. 创建新特征
df_stage2_train['cdi_mmi_interaction'] = df_stage2_train['cdi'] * df_stage2_train['mmi']
df_stage2_train['cdi_sq'] = df_stage2_train['cdi']**2
df_stage2_train['mmi_sq'] = df_stage2_train['mmi']**2

print("\n✅ Preprocessing and feature engineering complete for Stage 2 data.")

# 3. 准备训练数据 X 和 y
X_stage2 = df_stage2_train.drop(columns=["id", "label"]).values
y_stage2 = df_stage2_train["label"].values

# 4. 使用 SMOTE 处理类别不平衡
smote = SMOTE(random_state=42)
X_stage2_resampled, y_stage2_resampled = smote.fit_resample(X_stage2, y_stage2)
print("\nResampled dataset shape for Stage 2: %s" % pd.Series(y_stage2_resampled).value_counts().to_dict())

# 5. 训练第二阶段模型 (随机森林)
# 使用在另一个notebook中找到的最佳参数作为起点
best_params_rf = {
    'n_estimators': 2000,
    'max_depth': 20,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1
}
stage2_model = RandomForestClassifier(**best_params_rf)
stage2_model.fit(X_stage2_resampled, y_stage2_resampled)

print("\n✅ Stage 2 model (RandomForest) trained successfully!")

## 合并预测并生成提交文件

In [ ]:
print("🚀 Generating final predictions by combining Stage 1 and Stage 2 models.")

# 1. 对测试集进行第一阶段预测
X_test_stage1 = df_test[features_raw]
test_pred_stage1 = stage1_model.predict(X_test_stage1) # 预测是否为 label 0

# 2. 初始化最终预测结果
final_predictions = np.zeros(len(df_test), dtype=int)

# 3. 将第一阶段预测为 label 0 的样本直接赋值
label_0_indices = np.where(test_pred_stage1 == 1)[0]
final_predictions[label_0_indices] = 0
print(f"Found {len(label_0_indices)} samples predicted as Label 0 in test set.")

# 4. 对第一阶段预测为非 label 0 的样本，进行第二阶段预测
non_label_0_indices = np.where(test_pred_stage1 == 0)[0]
print(f"Found {len(non_label_0_indices)} samples to be classified by Stage 2 model.")

if len(non_label_0_indices) > 0:
    # a. 准备这些样本的数据
    df_test_stage2 = df_test.iloc[non_label_0_indices].copy()
    
    # b. 应用与训练时完全相同的预处理流程
    for col in feature_cols:
        df_test_stage2[col] = winsorize(df_test_stage2[col], limits=[0.01, 0.05])
    
    df_test_stage2['depth'] = np.log1p(df_test_stage2['depth'])
    
    # 使用在第二阶段训练数据上 fit 好的 scaler
    df_test_stage2[numerical_features] = scaler.transform(df_test_stage2[numerical_features])
    
    df_test_stage2['cdi_mmi_interaction'] = df_test_stage2['cdi'] * df_test_stage2['mmi']
    df_test_stage2['cdi_sq'] = df_test_stage2['cdi']**2
    df_test_stage2['mmi_sq'] = df_test_stage2['mmi']**2
    
    X_test_stage2 = df_test_stage2.drop(columns=["id"]).values
    
    # c. 使用第二阶段模型进行预测
    test_pred_stage2 = stage2_model.predict(X_test_stage2)
    
    # d. 将预测结果填充到最终预测数组中
    final_predictions[non_label_0_indices] = test_pred_stage2

# 5. 创建提交文件
submission_hierarchical = pd.DataFrame({
    "id": df_test["id"],
    "label": final_predictions
})
submission_hierarchical.to_csv("submission_hierarchical.csv", index=False)

print("\n✅ Hierarchical model predictions complete. Results saved to submission_hierarchical.csv")
print("\nSubmission file head:")
print(submission_hierarchical.head())
print("\nFinal prediction distribution:")
print(submission_hierarchical['label'].value_counts().sort_index())